<a href="https://colab.research.google.com/github/vicky1402/Spring_Info5731/blob/main/Rampelli_vivekraj_Exercise_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 5**

**This exercise aims to provide a comprehensive learning experience in text analysis and machine learning techniques, focusing on both text classification and clustering tasks.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
def load_reviews(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [line.strip().split(' ', 1) for line in file]
    labels, reviews = zip(*[(int(line[0]), line[1]) for line in data])
    return labels, reviews

train_targets, train_texts = load_reviews('/content/drive/MyDrive/5731/exercise09_datacollection/stsa-train.txt')
test_targets, test_texts = load_reviews('/content/drive/MyDrive/5731/exercise09_datacollection/stsa-test.txt')


***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## **Question 1 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text classification** as well as the performance evaluation. In addition, you are requried to conduct **10 fold cross validation** (https://scikit-learn.org/stable/modules/cross_validation.html) in the training.



The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.


**Algorithms:**

*   MultinominalNB
*   SVM
*   KNN
*   Decision tree
*   Random Forest
*   XGBoost
*   Word2Vec
*   BERT

**Evaluation measurement:**


*   Accuracy
*   Recall
*   Precison
*   F-1 score


In [12]:
import re

def clean_reviews(reviews):
    # Remove special characters, punctuation, and digits
    cleaned_reviews = [re.sub(r'[^A-Za-z\s]', '', review) for review in reviews]
    # Convert to lowercase
    cleaned_reviews = [review.lower() for review in cleaned_reviews]
    return cleaned_reviews

# Clean the training reviews
cleaned_train_reviews = clean_reviews(train_texts)
# Clean the test reviews
cleaned_test_reviews = clean_reviews(test_texts)

def tokenize_reviews(reviews):
    tokenized_reviews = [review.split() for review in reviews]
    return tokenized_reviews

# Tokenize the training reviews
tokenized_train_reviews = tokenize_reviews(cleaned_train_reviews)
# Tokenize the test reviews
tokenized_test_reviews = tokenize_reviews(cleaned_test_reviews)

def build_vocabulary(tokenized_reviews):
    vocabulary = set()
    for review in tokenized_reviews:
        vocabulary.update(review)
    return vocabulary

# Build vocabulary for training reviews
vocabulary_train = build_vocabulary(tokenized_train_reviews)
# Build vocabulary for test reviews
vocabulary_test = build_vocabulary(tokenized_test_reviews)

def encode_reviews(tokenized_reviews, vocabulary):
    word_to_index = {word: i for i, word in enumerate(vocabulary)}
    encoded_reviews = [[word_to_index[word] for word in review if word in word_to_index] for review in tokenized_reviews]
    return encoded_reviews

# Encode training reviews
encoded_train_reviews = encode_reviews(tokenized_train_reviews, vocabulary_train)
# Encode test reviews
encoded_test_reviews = encode_reviews(tokenized_test_reviews, vocabulary_test)

movie_reviews = list(zip(range(1, len(train_targets)+1), train_targets, train_texts))

# Print the movie reviews
for index, label, review in movie_reviews:
    sentiment = "positive" if label == 1 else "negative"
    print(f"Review {index} ({sentiment}): {review}")


Streaming output truncated to the last 5000 lines.
Review 1921 (negative): terrible .
Review 1922 (negative): throwing in everything except someone pulling the pin from a grenade with his teeth , windtalkers seems to have ransacked every old world war ii movie for overly familiar material .
Review 1923 (negative): chai 's structure and pacing are disconcertingly slack .
Review 1924 (positive): if you grew up on scooby -- you 'll love this movie .
Review 1925 (positive): enjoyably fast-moving , hard-hitting documentary .
Review 1926 (positive): if you come from a family that eats , meddles , argues , laughs , kibbitzes and fights together , then go see this delightful comedy .
Review 1927 (negative): the director mostly plays it straight , turning leys ' fable into a listless climb down the social ladder .
Review 1928 (negative): millions of dollars heaped upon a project of such vast proportions need to reap more rewards than spiffy bluescreen technique and stylish weaponry .
Review 192

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from gensim.models import Word2Vec
import numpy as np


# Split data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define models
models = {
    "MultinomialNB": MultinomialNB(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(),
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(),
    "XGBoost": XGBClassifier()
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Evaluating {model_name}:")
    if model_name == "Word2Vec":
        # Train Word2Vec model
        word2vec_model = Word2Vec(X_train, vector_size=100, window=5, min_count=1, workers=4)
        X_train_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in X_train])
        X_val_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in X_val])
        model.fit(X_train_embeddings, y_train)
    else:
        tfidf_vectorizer = TfidfVectorizer()
        X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
        X_val_tfidf = tfidf_vectorizer.transform(X_val)
        model.fit(X_train_tfidf, y_train)

    # Perform 10-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_tfidf if model_name != "Word2Vec" else X_train_embeddings, y_train, cv=kfold, scoring='accuracy')
    print(f"Mean Cross-Validation Accuracy: {np.mean(cv_scores):.4f} (±{np.std(cv_scores):.4f})")

    # Final evaluation on validation data
    if model_name == "Word2Vec":
        y_pred = model.predict(X_val_embeddings)
    else:
        y_pred = model.predict(X_val_tfidf)

    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    # Print validation metrics
    print("Validation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print()

    # Final evaluation on test data
    print("Final Evaluation on Test Data:")
    if model_name == "Word2Vec":
        X_test_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in X_test])
        y_pred_test = model.predict(X_test_embeddings)
    else:
        X_test_tfidf = tfidf_vectorizer.transform(X_test)
        y_pred_test = model.predict(X_test_tfidf)

    # Calculate test metrics
    accuracy_test = accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test)
    recall_test = recall_score(y_test, y_pred_test)
    f1_test = f1_score(y_test, y_pred_test)

    # Print test metrics
    print(f"Accuracy: {accuracy_test:.4f}")
    print(f"Precision: {precision_test:.4f}")
    print(f"Recall: {recall_test:.4f}")
    print(f"F1 Score: {f1_test:.4f}")
    print()


Evaluating MultinomialNB:
Mean Cross-Validation Accuracy: 0.7719 (±0.0164)
Validation Metrics:
Accuracy: 0.7762
Precision: 0.7454
Recall: 0.8534
F1 Score: 0.7957

Final Evaluation on Test Data:
Accuracy: 0.7832
Precision: 0.7438
Recall: 0.8836
F1 Score: 0.8077

Evaluating SVM:
Mean Cross-Validation Accuracy: 0.7595 (±0.0111)
Validation Metrics:
Accuracy: 0.7626
Precision: 0.7529
Recall: 0.7968
F1 Score: 0.7742

Final Evaluation on Test Data:
Accuracy: 0.7840
Precision: 0.7601
Recall: 0.8485
F1 Score: 0.8019

Evaluating KNN:
Mean Cross-Validation Accuracy: 0.7008 (±0.0202)
Validation Metrics:
Accuracy: 0.6968
Precision: 0.6923
Recall: 0.7314
F1 Score: 0.7113

Final Evaluation on Test Data:
Accuracy: 0.7095
Precision: 0.6986
Recall: 0.7672
F1 Score: 0.7313

Evaluating DecisionTree:
Mean Cross-Validation Accuracy: 0.5876 (±0.0219)
Validation Metrics:
Accuracy: 0.6065
Precision: 0.6136
Recall: 0.6201
F1 Score: 0.6169

Final Evaluation on Test Data:
Accuracy: 0.6105
Precision: 0.6173
Recall

In [14]:
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
import numpy as np

# Assuming you have train_texts and train_labels

# Split data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)
X_train_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in words.split() if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in X_train])
X_val_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in words.split() if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in X_val])

# Define XGBoost model
xgb_model = XGBClassifier()

# Train XGBoost model
xgb_model.fit(X_train_embeddings, y_train)

# Validation evaluation
y_pred = xgb_model.predict(X_val_embeddings)

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

# Print validation metrics
print("Word2Vec Validation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print()

# Final evaluation on test data
X_test_embeddings = np.array([np.mean([word2vec_model.wv[word] for word in words.split() if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in X_test])
y_pred_test = xgb_model.predict(X_test_embeddings)

# Calculate test metrics
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)

# Print test metrics
print("Word2Vec Test Metrics:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print()


Word2Vec Validation Metrics:
Accuracy: 0.5171
Precision: 0.5213
Recall: 0.6696
F1 Score: 0.5862

Word2Vec Test Metrics:
Accuracy: 0.5636
Precision: 0.5582
Recall: 0.7335
F1 Score: 0.6339



In [7]:

!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [16]:
# BERT MODEL
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(train_labels))

# Tokenization function for BERT
def tokenize_for_bert(reviews):
    # Ensure input is a list of strings
    reviews_list = reviews if isinstance(reviews, list) else list(reviews)
    return tokenizer(reviews_list, padding=True, truncation=True, max_length=128, return_tensors='pt')

# Prepare datasets
train_encodings = tokenize_for_bert(train_reviews)
test_encodings = tokenize_for_bert(test_reviews)

# Convert labels to tensor format
train_labels_tensor = torch.tensor(train_labels)
test_labels_tensor = torch.tensor(test_labels)

# Define a PyTorch dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Prepare datasets
train_dataset = TextDataset(train_reviews, train_labels_tensor, tokenizer)
test_dataset = TextDataset(test_reviews, test_labels_tensor, tokenizer)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Ensure at least one full pass over the data
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=50,  # Reduced to better fit smaller datasets
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch"  # Change to evaluate at the end of each epoch
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train and evaluate
trainer.train()





Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-7e20de793f3e>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss
1,1.298400,0.277515


<ipython-input-16-7e20de793f3e>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


NameError: name 'evaluation_results' is not defined

In [17]:

# Get predictions for the test dataset
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)
true_labels = test_labels_tensor.numpy()

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, zero_division=0)
recall = recall_score(true_labels, predicted_labels, zero_division=0)
f1 = f1_score(true_labels, predicted_labels, zero_division=0)

# Print evaluation metrics
print("BERT Test Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Store evaluation results if needed
evaluation_results = {
    'eval_accuracy': accuracy,
    'eval_recall': recall,
    'eval_precision': precision,
    'eval_f1': f1
}


<ipython-input-16-7e20de793f3e>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


BERT Test Metrics:
Accuracy: 0.9093904448105437
Precision: 0.891578947368421
Recall: 0.9317931793179318
F1 Score: 0.9112426035502957


## **Question 2 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text clustering**.

Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

**Apply the listed clustering methods to the dataset:**
*   K-means
*   DBSCAN
*   Hierarchical clustering
*   Word2Vec
*   BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [8]:
# Required Libraries
import pandas as pd
import numpy as np
import torch
from sklearn.cluster import KMeans, DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import linkage, fcluster, inconsistent
from sklearn.metrics import silhouette_score
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset

# Function Definitions
def load_reviews(file_path, sample_size=1000, random_seed=42):
    review_data = pd.read_csv(file_path)
    sampled_reviews = review_data.sample(n=sample_size, random_state=random_seed)
    return sampled_reviews

def vectorize_text(text_data):
    text_data['Reviews'].fillna('', inplace=True)
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    feature_matrix = tfidf_vectorizer.fit_transform(text_data['Reviews'])
    return feature_matrix

def perform_kmeans(feature_data, num_clusters, random_seed=42):
    kmeans_model = KMeans(n_clusters=num_clusters, random_state=random_seed)
    cluster_labels = kmeans_model.fit_predict(feature_data)
    return cluster_labels

def perform_dbscan(feature_data, epsilon=0.5, min_points=5):
    dbscan_model = DBSCAN(eps=epsilon, min_samples=min_points)
    cluster_labels = dbscan_model.fit_predict(feature_data)
    return cluster_labels

def perform_hierarchical_clustering(feature_data):
    linkage_matrix = linkage(feature_data.toarray(), method='ward')
    inconsistency_depth = 5
    inconsistency_scores = inconsistent(linkage_matrix, inconsistency_depth)
    distance_threshold = inconsistency_scores[-1, 3]
    cluster_labels = fcluster(linkage_matrix, distance_threshold, criterion='distance')
    return cluster_labels

def train_w2v_model(review_texts):
    sentences = review_texts.apply(lambda x: x.split())
    w2v_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)
    return w2v_model

def compute_word_embeddings(w2v_model, words):
    embeddings = []
    for word in words:
        embeddings.append(w2v_model.wv[word] if word in w2v_model.wv.key_to_index else np.zeros(w2v_model.vector_size))
    return np.mean(embeddings, axis=0) if embeddings else np.zeros(w2v_model.vector_size)

def infer_text_embeddings(w2v_model, review_texts):
    return np.array([compute_word_embeddings(w2v_model, text.split()) for text in review_texts])

def compute_bert_embeddings(text_reviews):
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    tokenized_inputs = bert_tokenizer(text_reviews.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=128)
    data_loader = DataLoader(TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask']), batch_size=32)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bert_model.to(device)
    all_embeddings = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_masks = [b.to(device) for b in batch]
            model_outputs = bert_model(input_ids, attention_mask=attention_masks)
            all_embeddings.append(model_outputs.last_hidden_state.mean(dim=1).cpu().numpy())
    return np.concatenate(all_embeddings, axis=0)

def evaluate_clusters(feature_data, cluster_labels):
    if np.min(cluster_labels) == -1:  # Handle noise (-1) labels from DBSCAN
        feature_data = feature_data[cluster_labels != -1]
        cluster_labels = cluster_labels[cluster_labels != -1]
    return silhouette_score(feature_data, cluster_labels) if len(set(cluster_labels)) > 1 else -1

# Main Execution Flow
if __name__ == "__main__":
    # Load and preprocess data
    reviews_data = load_reviews('/content/drive/MyDrive/5731/Amazon_Unlocked_Mobile.csv')
    text_features = vectorize_text(reviews_data)

    # Clustering
    kmeans_clusters = perform_kmeans(text_features, num_clusters=5)
    dbscan_clusters = perform_dbscan(text_features)
    hierarchical_clusters = perform_hierarchical_clustering(text_features)

    # Word2Vec Model
    w2v_model = train_w2v_model(reviews_data['Reviews'])
    w2v_embeddings = infer_text_embeddings(w2v_model, reviews_data['Reviews'])

    # BERT Embeddings
    bert_embeddings = compute_bert_embeddings(reviews_data['Reviews'])

    # Evaluate clustering
    print("K-means silhouette score:", evaluate_clusters(text_features, kmeans_clusters))
    print("DBSCAN silhouette score:", evaluate_clusters(text_features, dbscan_clusters))
    print("Hierarchical silhouette score:", evaluate_clusters(text_features, hierarchical_clusters))
    print("Word2Vec silhouette score:", evaluate_clusters(w2v_embeddings, kmeans_clusters))
    print("BERT silhouette score:", evaluate_clusters(bert_embeddings, kmeans_clusters))


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

K-means silhouette score: 0.040755872323074045
DBSCAN silhouette score: 0.9374054590092558
Hierarchical silhouette score: 0.050884591013098925
Word2Vec silhouette score: 0.08596438
BERT silhouette score: 0.09084683


**In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.**

**Write your response here:**

.The silhouette scores provide insights into the clustering and embedding performance. DBSCAN demonstrates the highest silhouette score, indicating dense, well-separated clusters, suitable for varying density data with noise. K-means yields a low silhouette score, suggesting less distinct clusters or suboptimal partitioning. Hierarchical clustering performs marginally better than K-means but falls short compared to DBSCAN, implying reasonable cluster separations. Word2Vec and BERT embeddings both surpass K-means and hierarchical clustering, with BERT slightly outperforming Word2Vec. This implies that both deep learning-based embedding methods capture more nuanced semantic information, potentially leading to more meaningful data representations conducive to clustering tasks, albeit with varying degrees of improvement.

.

.

.

.




# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.


**(Your submission will not be graded if this question is left unanswered)**



In [10]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
In this assignment all the models training are easy other than word2vec and bert models. Since both these models are large in size they take longer to run and more compute power is needed.
I learnt how to train most of the models in this class i still have to work on large models.
small models are comparitively easier prep the data, train, fit and test the accuracy of the models.




'''

'\nIn this assignment all the models training are easy other than word2vec and bert models. Since both these models are large in size they take longer to run and more compute power is needed.\nI learnt how to train most of the models in this class i still have to work on large models. \nsmall models are comparitively easier prep the data, train, fit and test the accuracy of the models.\n\n\n\n\n'